<a href="https://colab.research.google.com/github/tayyMem/ProjectGG/blob/main/load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers

from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# repo_name = "TayyMem/Grammar_Spelling_Correction"
repo_name = "TayyMem/ProjectGG_Testing3"

# Load model and tokenizer from Hugging Face
tokenizer = T5Tokenizer.from_pretrained(repo_name)
model = T5ForConditionalGeneration.from_pretrained(repo_name)

print("Model loaded successfully!")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Model loaded successfully!


https://huggingface.co/docs/transformers/v4.15.0/en/model_sharing#add-a-model-card

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# # Define the device
# # If a GPU is available, it will use the GPU. Otherwise, it will use the CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

example_sentence = [
    "He is there when I had finished having diner",
    "She go to school every day.",
    "The cloths he wear is given by me.",
    "he mistaked that it wre me.",
    "I comitted to this jobe and i planned it perfect",
    "This are the bestest clothes I have ever wated.",
]

#tokenizing the example sentence
testing_input = tokenizer(
    example_sentence,
    max_length=256,
    truncation=True,
    padding=True,
    return_tensors='pt').to(device)

#read more
testing_output = model.generate(
    input_ids= testing_input["input_ids"],
    attention_mask= testing_input["attention_mask"],
    max_length=256) # A way to predict data

corrected_sentences = [tokenizer.decode(output, skip_special_tokens=True) for output in testing_output]

for i in range(len(example_sentence)):
    print(f"❌ Incorrect: {example_sentence[i]}")
    print(f"✅ Corrected: {corrected_sentences[i]}\n")

❌ Incorrect: He is there when I had finished having diner
✅ Corrected: He is there when i had finished having dinner

❌ Incorrect: She go to school every day.
✅ Corrected: She goes to school every day.

❌ Incorrect: The cloths he wear is given by me.
✅ Corrected: The cloths he wear is given by me.

❌ Incorrect: he mistaked that it wre me.
✅ Corrected: he mistaken that it was me.

❌ Incorrect: I comitted to this jobe and i planned it perfect
✅ Corrected: I committed to this job and i planned it perfect

❌ Incorrect: This are the bestest clothes I have ever wated.
✅ Corrected: These are the best clothes i have ever worn.

